<a href="https://colab.research.google.com/github/NAKDO/machine-learning/blob/main/%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 1 201935213 김낙도

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#데이터 불러오기 & 전처리

In [25]:
# 데이터 불러오기
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")

# 훈련 데이터와 테스트 데이터를 합쳐서 데이터 전처리를 한 번에 수행
all_data = pd.concat([train_data, test_data])

# NaN 값 처리
1.Embarked와 Fare 열의 NaN 값 처리

SimpleImputer를 사용하여 'Embarked'와 'Fare' 열의 NaN 값을 대체했습니다. strategy='most_frequent'로 설정하여 'Embarked' 열의 NaN 값을 가장 빈번하게 나타나는 값으로 대체하고, 'Fare' 열의 NaN 값을 역시 가장 빈번하게 나타나는 값으로 대체했습니다.

2.Age 열의 NaN 값 처리:

SimpleImputer를 사용하여 'Age' 열의 NaN 값을 중간값(median)으로 대체했습니다. strategy='median'으로 설정하여 'Age' 열의 NaN 값을 중간값으로 대체하였습니다.

In [26]:
# NaN 값 처리
imputer = SimpleImputer(strategy='most_frequent')
all_data['Embarked'] = imputer.fit_transform(all_data['Embarked'].values.reshape(-1, 1))
all_data['Fare'] = imputer.fit_transform(all_data['Fare'].values.reshape(-1, 1))

In [27]:
# 나이는 중간값으로 대체
age_imputer = SimpleImputer(strategy='median')
all_data['Age'] = age_imputer.fit_transform(all_data['Age'].values.reshape(-1, 1))

# Cabin 특성은 너무 많은 결측값이 있어 제외
all_data.drop('Cabin', axis=1, inplace=True)

# 특성 공학: 이름에서 호칭 추출
all_data['Title'] = all_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
all_data['Title'] = all_data['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
all_data['Title'] = all_data['Title'].replace('Mlle', 'Miss')
all_data['Title'] = all_data['Title'].replace('Ms', 'Miss')
all_data['Title'] = all_data['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
all_data['Title'] = all_data['Title'].map(title_mapping)
all_data['Title'] = all_data['Title'].fillna(0)

# 가족 크기 파생 특성 생성
all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1

train.csv에 Sex변수가 Sex_male로 되어 있어서 Sex_male로 바꿔주었음.

In [28]:
# 필요한 열 선택
selected_features = ['Pclass_2', 'Pclass_3', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S', 'Sex_male']

# 원핫 인코딩
all_data = pd.get_dummies(all_data, columns=['Sex', 'Pclass', 'Embarked'], drop_first=True)

## 데이터 분할

In [29]:
# 데이터 분할
X_train = all_data[:len(train_data)]
X_test = all_data[len(train_data):]
y_train = train_data['Survived']

# 훈련 및 테스트 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=201935213)

### 표준 스케일링

In [30]:
label_encoder = LabelEncoder()
X_train['Sex_male'] = label_encoder.fit_transform(X_train['Sex_male'])
X_val['Sex_male'] = label_encoder.transform(X_val['Sex_male'])
X_test['Sex_male'] = label_encoder.transform(X_test['Sex_male'])

# 표준 스케일링
scaler = StandardScaler()
X_train[selected_features] = scaler.fit_transform(X_train[selected_features])
X_val[selected_features] = scaler.transform(X_val[selected_features])
X_test[selected_features] = scaler.transform(X_test[selected_features])

<ipython-input-30-0beca1788ae8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Sex_male'] = label_encoder.transform(X_test['Sex_male'])
<ipython-input-30-0beca1788ae8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[selected_features] = scaler.transform(X_test[selected_features])


# SGD 분류
SGDClassifier 모델을 생성하고 훈련 데이터로 모델을 훈련시키며, 훈련 및 검증 데이터에 대한 정확도를 출력합니다. 이를 통해 모델의 성능을 확인할 수 있습니다. 나중에 하이퍼파라미터 최적화를 위한 그리드 서치 또는 랜덤 서치를 적용할 수 있습니다.

In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# SGDClassifier 모델 생성
sgd_model = SGDClassifier(random_state=201935213)

# 모델 훈련
sgd_model.fit(X_train[selected_features], y_train)

# 훈련 데이터에 대한 예측
train_predictions = sgd_model.predict(X_train[selected_features])

# 검증 데이터에 대한 예측
val_predictions = sgd_model.predict(X_val[selected_features])

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
val_accuracy = accuracy_score(y_val, val_predictions)

print("훈련 데이터 정확도:", train_accuracy)
print("검증 데이터 정확도:", val_accuracy)



훈련 데이터 정확도: 0.7471910112359551
검증 데이터 정확도: 0.8156424581005587


# 결정트리
결정 트리 분류기 모델을 생성하고 훈련 데이터로 모델을 훈련시키며, 훈련 및 검증 데이터에 대한 정확도를 출력합니다. 이를 통해 모델의 성능을 확인할 수 있습니다. 나중에 하이퍼파라미터 최적화를 위해 그리드 서치 또는 랜덤 서치를 적용할 수 있습니다.

In [20]:
from sklearn.tree import DecisionTreeClassifier

# 결정 트리 분류기 모델 생성
decision_tree_model = DecisionTreeClassifier(random_state=201935213)

# 모델 훈련
decision_tree_model.fit(X_train[selected_features], y_train)

# 훈련 데이터에 대한 예측
train_predictions = decision_tree_model.predict(X_train[selected_features])

# 검증 데이터에 대한 예측
val_predictions = decision_tree_model.predict(X_val[selected_features])

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
val_accuracy = accuracy_score(y_val, val_predictions)

print("훈련 데이터 정확도:", train_accuracy)
print("검증 데이터 정확도:", val_accuracy)


훈련 데이터 정확도: 0.9803370786516854
검증 데이터 정확도: 0.8156424581005587


# 랜덤포래스트
랜덤 포레스트 분류기 모델을 생성하고 훈련 데이터로 모델을 훈련시키며, 훈련 및 검증 데이터에 대한 정확도를 출력합니다. 이를 통해 모델의 성능을 확인할 수 있습니다. 나중에 하이퍼파라미터 최적화를 위해 그리드 서치 또는 랜덤 서치를 적용할 수 있습니다

In [21]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 분류기 모델 생성
random_forest_model = RandomForestClassifier(random_state=201935213)

# 모델 훈련
random_forest_model.fit(X_train[selected_features], y_train)

# 훈련 데이터에 대한 예측
train_predictions = random_forest_model.predict(X_train[selected_features])

# 검증 데이터에 대한 예측
val_predictions = random_forest_model.predict(X_val[selected_features])

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
val_accuracy = accuracy_score(y_val, val_predictions)

print("훈련 데이터 정확도:", train_accuracy)
print("검증 데이터 정확도:", val_accuracy)


훈련 데이터 정확도: 0.9803370786516854
검증 데이터 정확도: 0.8435754189944135


# 히스토그램기반 그래디언트 부스팅
히스토그램 기반 그래디언트 부스팅 분류기 모델을 생성하고 훈련 데이터로 모델을 훈련시키며, 훈련 및 검증 데이터에 대한 정확도를 출력합니다. 이를 통해 모델의 성능을 확인할 수 있습니다. 나중에 하이퍼파라미터 최적화를 위해 그리드 서치 또는 랜덤 서치를 적용할 수 있습니다.

In [23]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

# 히스토그램 기반 그래디언트 부스팅 분류기 모델 생성
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=201935213)

# 모델 훈련
hist_gradient_boosting_model.fit(X_train[selected_features], y_train)

# 훈련 데이터에 대한 예측
train_predictions = hist_gradient_boosting_model.predict(X_train[selected_features])

# 검증 데이터에 대한 예측
val_predictions = hist_gradient_boosting_model.predict(X_val[selected_features])

# 정확도 출력
train_accuracy = accuracy_score(y_train, train_predictions)
val_accuracy = accuracy_score(y_val, val_predictions)

print("훈련 데이터 정확도:", train_accuracy)
print("검증 데이터 정확도:", val_accuracy)


훈련 데이터 정확도: 0.9396067415730337
검증 데이터 정확도: 0.8715083798882681
